<a href="https://colab.research.google.com/github/jang-jinyeol/Project/blob/master/Resnet18_classifier_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount to your google drive

#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
! ls 'drive/My Drive/Rosa_classifier'

train_data.npy	val_data.npy


In [ ]:
import numpy as np
from torch.utils.data import DataLoader,Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import os
import copy

In [ ]:
train_data = np.load('drive/My Drive/Rosa_classifier/train_data.npy', allow_pickle=True)
val_data = np.load('drive/My Drive/Rosa_classifier/val_data.npy', allow_pickle=True)

In [ ]:
train_data[0][0]

array([[[ 0.82755375,  0.810429  ,  0.810429  , ...,  0.5878072 ,
          0.6563062 ,  0.70768046],
        [ 0.810429  ,  0.79330426,  0.79330426, ...,  0.5535577 ,
          0.60493195,  0.63918144],
        [ 0.79330426,  0.79330426,  0.79330426, ...,  0.45080918,
          0.5193082 ,  0.57068247],
        ...,
        [ 0.34806067,  0.38231018,  0.38231018, ...,  0.09118938,
          0.12543888,  0.15968838],
        [ 0.3651854 ,  0.3651854 ,  0.3651854 , ...,  0.12543888,
          0.14256364,  0.15968838],
        [ 0.3651854 ,  0.34806067,  0.34806067, ...,  0.14256364,
          0.14256364,  0.12543888]],

       [[ 0.8004202 ,  0.78291327,  0.78291327, ...,  0.5378152 ,
          0.57282925,  0.6078432 ],
        [ 0.78291327,  0.76540625,  0.76540625, ...,  0.50280124,
          0.5378152 ,  0.57282925],
        [ 0.76540625,  0.76540625,  0.76540625, ...,  0.3977592 ,
          0.4677872 ,  0.5203082 ],
        ...,
        [ 0.03011205,  0.04761905,  0.06512605, ...,  

In [ ]:
class Data(Dataset):
  def __init__(self, datafile):
    self.datafile = datafile

  def __len__(self):
    return self.datafile.shape[0]

  def __getitem__(self, idx):
    img = self.datafile[idx][0]
    y = int(self.datafile[idx][1])
    if y == 0:
      lbl = torch.tensor([1.0,0.0])
    if y == 1:
      lbl = torch.tensor([0.0,1.0])

    return img, lbl

In [ ]:
#train_data_class.__getitem__(0)

In [ ]:
# Make data class
train_data_class = Data(train_data)
val_data_class = Data(val_data)

In [ ]:
# Make data loader
train_loader = DataLoader(train_data_class, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data_class, batch_size=32, shuffle=True)

In [ ]:
dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': train_data.shape[0], 'val': val_data.shape[0]}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Build Model

In [ ]:
## Model as feature extractor
model_conv = models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Sequential(nn.Linear(num_ftrs, 2), torch.nn.Sigmoid())

criterion = nn.MSELoss()

# Observe that only parameters of final layer are being optimized as opposed to before.
#optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)
optimizer_conv = optim.Adam(model_conv.fc.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [ ]:
# Testing with one batch

img, lbl = next(iter(dataloaders['val']))

outputs = model_conv(img)
value, preds = torch.max(outputs, 1)
loss = criterion(outputs, lbl)

print(loss)

tensor(0.0388, grad_fn=<MseLossBackward>)


In [ ]:
def train_model(model, optimizer,criterion, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 1.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    value, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                
                running_corrects += torch.sum(preds == torch.argmax(labels,1)) #labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_conv = train_model(model_conv,  optimizer_conv, criterion,  exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 0.0672 Acc: 0.9814
val Loss: 0.0366 Acc: 1.0000

Epoch 1/24
----------
train Loss: 0.0366 Acc: 0.9912
val Loss: 0.0247 Acc: 1.0000

Epoch 2/24
----------
train Loss: 0.0263 Acc: 0.9912
val Loss: 0.0164 Acc: 1.0000

Epoch 3/24
----------
train Loss: 0.0207 Acc: 0.9967
val Loss: 0.0110 Acc: 1.0000

Epoch 4/24
----------
train Loss: 0.0191 Acc: 0.9934
val Loss: 0.0175 Acc: 1.0000

Epoch 5/24
----------
train Loss: 0.0182 Acc: 0.9923
val Loss: 0.0056 Acc: 1.0000

Epoch 6/24
----------
train Loss: 0.0166 Acc: 0.9956
val Loss: 0.0085 Acc: 1.0000

Epoch 7/24
----------
train Loss: 0.0134 Acc: 0.9989
val Loss: 0.0065 Acc: 1.0000

Epoch 8/24
----------
train Loss: 0.0163 Acc: 0.9967
val Loss: 0.0057 Acc: 1.0000

Epoch 9/24
----------
train Loss: 0.0128 Acc: 0.9978
val Loss: 0.0078 Acc: 1.0000

Epoch 10/24
----------
train Loss: 0.0122 Acc: 0.9978
val Loss: 0.0056 Acc: 1.0000

Epoch 11/24
----------
train Loss: 0.0144 Acc: 0.9978
val Loss: 0.0061 Acc: 1.0000

Ep

In [ ]:
torch.save(model_conv.state_dict(), '/content/drive/My Drive/dataset/model-resnet18-2.pth')